<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Tabular_data_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q langchain
!pip install -q openai
!pip install -q langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


In [16]:
#%%capture
import langchain
import pandas as pd
import bs4
import openai
import tabulate

In [19]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI

In [33]:
# Get your free API key from: www.cohere.com
openai_key = ""
llm = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key=openai_key)

In [8]:
# Here we load a pandas DataFrame with around top-5000 movies
#df = pd.read_parquet("https://huggingface.co/datasets/Cohere/movies/resolve/main/movies.parquet")
df = pd.read_csv("/content/results_laptop_proj.csv")
df

,concept,relevance,score_driver,rating,how much on average does mentioning this concept impact the final rating of a feedback,positive sentiment percentage,neutral sentiment percentage,negative sentiment percentage,positive_feedback,netral_feedback,negative_feedback,total_sentiment
0,laptop,1230.000000,Rating,4.148432,-0.314338,0.645296,0.154007,0.200697,926,221,288,1435
1,appearance,777.551020,Rating,4.929164,0.466394,0.905803,0.042954,0.051243,1202,57,68,1327
2,smooth,547.000000,Rating,4.845258,0.382488,0.971714,0.009983,0.018303,584,6,11,601
3,keyboard,538.000000,Rating,4.391557,-0.071213,0.669578,0.066958,0.263464,460,46,181,687
4,Packaging protection,513.000000,Rating,4.957115,0.494345,0.838207,0.066277,0.095517,430,34,49,513
...,...,...,...,...,...,...,...,...,...,...,...,...
495,im,29.948408,Rating,4.646154,0.183384,0.646154,0.215385,0.138462,42,14,9,65
496,not recommend,29.687134,Rating,2.733333,-1.729437,0.022222,0.000000,0.977778,1,0,44,45
497,not affect,29.606342,Rating,4.026316,-0.436454,0.500000,0.342105,0.157895,19,13,6,38
498,meet the needs,29.573059,Rating,4.918033,0.455263,0.983607,0.016393,0.000000,60,1,0,61


In [34]:
p_agent = create_pandas_dataframe_agent(llm=llm,df=df,verbose=True,allow_dangerous_code=True)

In [35]:
p_agent.run("give me all concepts with the relevance above 500")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe by the relevance column, keeping only rows where the value is above 500.
Action: python_repl_ast
Action Input: df[df['relevance'] > 500]                concept    relevance score_driver    rating  \
0                laptop  1230.000000       Rating  4.148432   
1            appearance   777.551020       Rating  4.929164   
2                smooth   547.000000       Rating  4.845258   
3              keyboard   538.000000       Rating  4.391557   
4  Packaging protection   513.000000       Rating  4.957115   
5                screen   500.877984       Rating  4.288636   

   how much on average does mentioning this concept impact the final rating of a feedback  \
0                                          -0.314338                                        
1                                           0.466394                                        
2                                           0.382488           

'The concepts with relevance above 500 are: laptop, appearance, smooth, keyboard, Packaging protection, and screen.'

In [36]:
results = p_agent.run("give me all concepts with the relevance above 500")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe based on the relevance column and then select the concept column
Action: python_repl_ast
Action Input: df[df['relevance'] > 500]['concept']0                  laptop
1              appearance
2                  smooth
3                keyboard
4    Packaging protection
5                  screen
Name: concept, dtype: object I now know the final answer
Final Answer: ['laptop', 'appearance', 'smooth', 'keyboard', 'Packaging protection', 'screen']

> Finished chain.


In [37]:
results

"['laptop', 'appearance', 'smooth', 'keyboard', 'Packaging protection', 'screen']"